In [1]:
import msprime
import copy


### Question: What is the average tMRCA for sex biased populations experiencing a bottleneck? 
* Do the tMRCAs fall within the reduced epoch?
* Can they partially explain patterns of variation observed? 

__Step 1: Create Demographic History__

In [2]:
BSRS = msprime.Demography()
BSRS.add_population(name="A", initial_size=10000)
BSRS.add_population_parameters_change(time=500, initial_size = 500)
BSRS.add_population_parameters_change(time=1000, initial_size = 10000)

BSRS

Demography(populations=[Population(initial_size=10000, growth_rate=0, name='A', description='', extra_metadata={}, default_sampling_time=None, initially_active=None, id=0)], events=[PopulationParametersChange(time=500, initial_size=500, growth_rate=None, population=-1), PopulationParametersChange(time=1000, initial_size=10000, growth_rate=None, population=-1)], migration_matrix=array([[0.]]))

__Step 2: Bring in function to adjust Ne based on BSR__

In [3]:
def adjust_input_params_A(pf, Demo):
    """
    This function changes all population sizes of the given demographic history based on the provided BSR. 
    Equation identical to that used in HTSimulate and only should be used for simulating the Autosome.
    """
    aut_var = 4*pf*(1-pf)
    Demo_new = copy.deepcopy(Demo)
    
    for pop in Demo_new.values():
        pop.initial_size *= aut_var
    for event in Demo_new.events:
        if event.initial_size == None:
            continue
        event.initial_size *= aut_var
    return Demo_new
    
def adjust_input_params_X(pf, Demo):
    """
    This function changes all population sizes of the given demographic history based on the provided BSR. 
    Equation identical to that used in HTSimulate and only should be used for simulating the X chromosome.
    """
    x_var = (9 * pf * (1-pf))/(2 * (2-pf))
    Demo_new = copy.deepcopy(Demo)
    
    for pop in Demo_new.values():
        pop.initial_size *= x_var
    for event in Demo_new.events:
        if event.initial_size == None:
            continue
        event.initial_size *= x_var
    return Demo_new

def get_rec_X(pf, rec_rate):
    """
    Because the recombination changes with different BSRs for the X, this function will adjust the given rate
    based on the provided pf. 
    """
    new_rate = (2*pf)/(1+pf) * rec_rate
    return new_rate 

__Step 3: Simulate some tree sequences for variable pfs__

Autosome:

In [4]:

A_Sim_01 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2,
                                demography = adjust_input_params_A(0.1,BSRS), num_replicates=5000)

A_Sim_015 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2,
                                demography = adjust_input_params_A(0.15,BSRS), num_replicates=5000)

A_Sim_02 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2,
                                 demography = adjust_input_params_A(0.2,BSRS), num_replicates=5000)

A_Sim_08 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2,
                             demography = adjust_input_params_A(0.8,BSRS), num_replicates=5000)

A_Sim_085 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2,
                             demography = adjust_input_params_A(0.85,BSRS), num_replicates=5000)

A_Sim_09 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2,
                             demography = adjust_input_params_A(0.9,BSRS), num_replicates=5000)


In [6]:
tMRCAs = {'0.1':[], '0.15':[], '0.2':[], '0.8':[], '0.85':[], '0.9':[]}
Branch_Lens = {'0.1':[], '0.15':[], '0.2':[], '0.8':[], '0.85':[], '0.9':[]}

for ts in A_Sim_01:
    for tree in ts.trees():
        tMRCAs['0.1'].append(tree.time(tree.root))
        Branch_Lens['0.1'].append(tree.total_branch_length)

for ts in A_Sim_015:
    for tree in ts.trees():
        tMRCAs['0.15'].append(tree.time(tree.root))
        Branch_Lens['0.15'].append(tree.total_branch_length)

for ts in A_Sim_02:    
    for tree in ts.trees():
        tMRCAs['0.2'].append(tree.time(tree.root))
        Branch_Lens['0.2'].append(tree.total_branch_length)
        
for ts in A_Sim_08:    
    for tree in ts.trees():
        tMRCAs['0.8'].append(tree.time(tree.root))
        Branch_Lens['0.8'].append(tree.total_branch_length)
  
for ts in A_Sim_085:
    for tree in ts.trees():
        tMRCAs['0.85'].append(tree.time(tree.root))
        Branch_Lens['0.85'].append(tree.total_branch_length)
        
for ts in A_Sim_09: 
    for tree in ts.trees():
        tMRCAs['0.9'].append(tree.time(tree.root))
        Branch_Lens['0.9'].append(tree.total_branch_length)


In [7]:
avg_tMRCAs = {'0.1':[], '0.15':[], '0.2':[], '0.8':[], '0.85':[], '0.9':[]}
avg_Branch_Lens = {'0.1':[], '0.15':[], '0.2':[], '0.8':[], '0.85':[], '0.9':[]}

for key in tMRCAs.keys():
    avg_tMRCAs[key].append(sum(tMRCAs[key]) / len(tMRCAs[key]))
    avg_Branch_Lens[key].append(sum(Branch_Lens[key]) / len(Branch_Lens[key]))
    
avg_tMRCAs

{'0.1': [5346.70786066362],
 '0.15': [10340.046417703195],
 '0.2': [14874.051625226277],
 '0.8': [15303.991960499101],
 '0.85': [10194.493008825964],
 '0.9': [5398.145848146192]}

In [8]:
avg_Branch_Lens

{'0.1': [18470.79068127841],
 '0.15': [30297.21143614607],
 '0.2': [41493.976113634875],
 '0.8': [42452.71812685504],
 '0.85': [30021.967210592586],
 '0.9': [18516.247468234727]}

X chromosome:

In [10]:

X_Sim_02 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2, 
                                demography = adjust_input_params_X(0.2,BSRS), num_replicates = 5000)

X_Sim_025 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2, 
                                demography = adjust_input_params_X(0.25,BSRS), num_replicates = 5000)

X_Sim_03 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2, 
                                demography = adjust_input_params_X(0.3,BSRS), num_replicates = 5000)

X_Sim_08 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2, 
                                demography = adjust_input_params_X(0.8,BSRS), num_replicates = 5000)

X_Sim_085 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2, 
                                demography = adjust_input_params_X(0.85,BSRS), num_replicates = 5000)

X_Sim_09 = msprime.sim_ancestry(samples = 10, sequence_length = 10000, ploidy=2, 
                                demography = adjust_input_params_X(0.9,BSRS), num_replicates = 5000)

In [11]:
tMRCAs = {'0.2':[], '0.25':[], '0.3':[], '0.8':[], '0.85':[], '0.9':[]}
Branch_Lens = {'0.2':[], '0.25':[], '0.3':[], '0.8':[], '0.85':[], '0.9':[]}

for ts in X_Sim_02:
    for tree in ts.trees():
        tMRCAs['0.2'].append(tree.time(tree.root))
        Branch_Lens['0.2'].append(tree.total_branch_length)

for ts in X_Sim_025:
    for tree in ts.trees():
        tMRCAs['0.25'].append(tree.time(tree.root))
        Branch_Lens['0.25'].append(tree.total_branch_length)

for ts in X_Sim_03:    
    for tree in ts.trees():
        tMRCAs['0.3'].append(tree.time(tree.root))
        Branch_Lens['0.3'].append(tree.total_branch_length)
        
for ts in X_Sim_08:    
    for tree in ts.trees():
        tMRCAs['0.8'].append(tree.time(tree.root))
        Branch_Lens['0.8'].append(tree.total_branch_length)
  
for ts in X_Sim_085:
    for tree in ts.trees():
        tMRCAs['0.85'].append(tree.time(tree.root))
        Branch_Lens['0.85'].append(tree.total_branch_length)
        
for ts in X_Sim_09: 
    for tree in ts.trees():
        tMRCAs['0.9'].append(tree.time(tree.root))
        Branch_Lens['0.9'].append(tree.total_branch_length)


In [12]:
avg_tMRCAs = {'0.2':[], '0.25':[], '0.3':[], '0.8':[], '0.85':[], '0.9':[]}
avg_Branch_Lens = {'0.2':[], '0.25':[], '0.3':[], '0.8':[], '0.85':[], '0.9':[]}

for key in tMRCAs.keys():
    avg_tMRCAs[key].append(sum(tMRCAs[key]) / len(tMRCAs[key]))
    avg_Branch_Lens[key].append(sum(Branch_Lens[key]) / len(Branch_Lens[key]))
    
avg_tMRCAs

{'0.2': [6562.577642995768],
 '0.25': [9352.085706223004],
 '0.3': [11982.319212216398],
 '0.8': [13666.231118292206],
 '0.85': [9948.775129600877],
 '0.9': [5596.433949250502]}

In [13]:
avg_Branch_Lens

{'0.2': [21318.125355218275],
 '0.25': [28008.882644606525],
 '0.3': [34350.63090427721],
 '0.8': [38356.20390457532],
 '0.85': [29355.858713731766],
 '0.9': [19034.059723831702]}

If a population's common ancestor occurs < 10,000 generations ago, the strange patterns in LD are observed. All of these trees are rooted prior to the reduction in size, but the patterns still emerge... What is going on? 

Compare internal vs external branch lengths? 